## 成交量(volume)预测
步骤(对于每次预测)：
1. 近一年历史数据训练模型
2. 近五日数据作为预测下一天的输入

注："一年"和"五日"可以作为参数调整

算法：
* 线性回归
* 神经网络

结果：

| date | real | linear | nn |
|------|--------|----------------|----------------|
|     |     |           |           |
|     |     |           |           |
|     |     |           |           |
|     |     |           |           |
|     |     |           |           |
|     |     |           |           |


In [1]:
import MySQLdb, datetime
import pandas as pd
import numpy as np

conn = MySQLdb.connect(host='219.224.169.45', user='lizimeng', passwd='codegeass', db='commodity')
cursor = conn.cursor()
sql = "select date,T1.code,sum(volume) from contract_daily T1,code2exchange T2 where T2.exchange='SHFE' and T1.code=T2.code and date>='20140101' \
group by date,T1.code order by date asc"
cursor.execute(sql)
df = pd.DataFrame(data=list(cursor.fetchall()), columns=['date','code','volume'])
print df.loc[:10]

          date code   volume
0   2014-01-02   ag  1621582
1   2014-01-02   al    16736
2   2014-01-02   au   118878
3   2014-01-02   bu    24404
4   2014-01-02   cu   325324
5   2014-01-02   fu        0
6   2014-01-02   pb     2544
7   2014-01-02   rb  1108358
8   2014-01-02   ru   417790
9   2014-01-02   wr       50
10  2014-01-02   zn    65528


In [ ]:
import MySQLdb, datetime
import numpy as np
from sklearn.neural_network import MLPRegressor

conn = MySQLdb.connect(host='219.224.169.45', user='lizimeng', passwd='codegeass', db='commodity', charset='utf8')
cursor = conn.cursor()

day1 = datetime.date(2014,1,1)
day2 = datetime.date.today() - datetime.timedelta(1)

commodities = []
days = []

cursor.execute('select distinct code from contract_daily where date between %s and %s', (day1,day2))
for row in cursor.fetchall():
    commodities.append(row)
    
cursor.execute('select distinct day from trade_day where day between %s and %s', (day1,day2))
for row in cursor.fetchall():
    days.append(row[0])
    
data = np.zeros((len(commodities), len(days)))

i = 0
for code in commodities:
    cursor.execute('select sum(volume) from contract_daily where code=%s and date between %s and %s group by date order by date asc', (code,day1,day2))
    j = 0
    for row in cursor.fetchall():
        data[i,j] = row[0]
        j += 1
    i += 1

cursor.close()
conn.close()

In [ ]:
# 切分训练集和测试集合，2017年1月1日之前为训练集，之后为测试集
point = 0
while days[point]<=datetime.date(2016,12,31):
    point += 1

In [ ]:
nn = MLPRegressor(hidden_layer_sizes=(200,200), activation='logistic', max_iter=10**6)
# 训练神经网络
X = []
Y = []
for i in range(point-5):
    X.append(data[:,i:i+5].reshape(len(commodities)*5).tolist())
    Y.append(data[:,i+5])
nn.fit(X,Y)

In [ ]:
# 测试神经网络
X = []
Y = []
for i in range(point, len(days)-5):
    X.append(data[:,i:i+5].reshape(len(commodities)*5).tolist())
    Y.append(data[:,i+5])
print nn.score(X,Y)

In [ ]:
import datetime
import numpy as np
import pandas as pd

In [ ]:
df = pd.Series(['hello','world','!'], dtype='string')

In [ ]:
df[0]

In [ ]:
np.dtype(str)